### Web scraping : Extracting books information

This notebook extract information from http://books.toscrape.com.

It uses **httpx** to perform asynchronous request on shop and book page

**BeautifulSoup** is later used to parse the html responses.

Extracted information are then saved in a .csv file.

In [12]:
# https://www.python-httpx.org/
# !pip install httpx
# https://beautiful-soup-4.readthedocs.io/en/latest/#
# !pip install beautifulsoup4
# https://github.com/alexmojaki/sorcery
# !pip install sorcery

In [13]:
from bs4 import BeautifulSoup
import httpx
import asyncio
from sorcery import dict_of
import pandas as pd

# optional
from rich import print

import re

In [14]:
root = 'http://books.toscrape.com/catalogue/'
base_url = 'http://books.toscrape.com/catalogue/page-{}.html'
shop_link = [base_url.format(i) for i in range(1,51)]

async with httpx.AsyncClient() as client:
    tasks = [client.get(url) for url in shop_link]
    reqs = await asyncio.gather(*tasks)
 
# Checking reqs status
failed = len([r for r in reqs if r.status_code != 200])
print(f'{len(reqs)} requests completed. {failed} failures')

50 requests completed. 0 failures

In [15]:
soups = [BeautifulSoup(r.text, 'html.parser') for r in reqs]
# Extracting links of book 
books_link = []

for i, soup in enumerate(soups):
    books_tag = soup.select('div.image_container > a')
    for book_tag in books_tag:
        books_link.append(root + str(book_tag['href']))
    # print(f'Extracted {len(books_tag)} books links from {shop_link[i]}')
    
print(f'Total book links extracted: {len(books_link)}')

Total book links extracted: 1000

In [16]:
# Extracting book page

async with httpx.AsyncClient() as client:
    tasks = [client.get(url) for url in books_link]
    reqs = await asyncio.gather(*tasks)
    
# Checking reqs status   
failed = len([r for r in reqs if r.status_code != 200])
print(f'{len(reqs)} requests completed. {failed} failures')

1000 requests completed. 0 failures

### Extracting books information

- Title - as primary key
- Book genre
- Book description
- Price
- Stock
- Rating
- Book cover url

In [17]:
soups = [BeautifulSoup(r.text, 'html.parser') for r in reqs]
books = dict()

def extract_book_info(soup):
    title = soup.select('div.product_main > h1')[0].text
    category = soup.select('ul.breadcrumb > li:nth-child(3) > a')[0].text
    if soup.select('article.product_page > p'):
        description = soup.select('article.product_page > p')[0].text
    else : 
        description = None
    price = soup.select('div.product_main > p.price_color')[0].text
    stock = soup.select('div.product_main > p.instock.availability')[0].text
    url = books_link[i]
    # Could be handled better but this 🤷‍♀️ work :  
    cover_url = 'http://books.toscrape.com/' + soup.select('div.thumbnail > div > div > img')[0]['src'][6:]
    
    rating = soup.select('p.star-rating')[0]['class'][1]
    rating_dict = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
    rating = rating_dict.get(rating, None)
    
    
    # Search digits in stock
    stock = re.search(r'\d+', stock)
    if stock:
        stock = int(stock.group(0))
    else: 
        stock = 0
    
    book = dict_of(category, rating, description, price, stock, url, cover_url)
    
    books[title] = book
    pass

soups = [BeautifulSoup(r.text, 'html.parser') for r in reqs]
for soup in soups:
    extract_book_info(soup)
    
print(f'Total book information extracted: {len(books_link)}')

Total book information extracted: 1000

In [18]:
# Some random samples of books extracted
df = pd.DataFrame(books).T
df = df.reset_index(names = 'title')
df.sample(5)

,title,category,rating,description,price,stock,url,cover_url
119,"Avatar: The Last Airbender: Smoke and Shadow, ...",Fantasy,2,Children are disappearing in the Fire Nation c...,£28.09,16,http://books.toscrape.com/catalogue/unicorn-tr...,http://books.toscrape.com/media/cache/9f/25/9f...
978,"Fruits Basket, Vol. 1 (Fruits Basket #1)",Sequential Art,5,A family with an ancient curse...And the girl ...,£40.28,1,http://books.toscrape.com/catalogue/unicorn-tr...,http://books.toscrape.com/media/cache/03/c5/03...
655,"The Demon Prince of Momochi House, Vol. 4 (The...",Sequential Art,2,"On her sixteenth birthday, orphan Himari Momoc...",£27.88,4,http://books.toscrape.com/catalogue/unicorn-tr...,http://books.toscrape.com/media/cache/20/99/20...
40,Slow States of Collapse: Poems,Poetry,3,The eagerly anticipated debut from one of Cana...,£57.31,17,http://books.toscrape.com/catalogue/unicorn-tr...,http://books.toscrape.com/media/cache/db/ac/db...
800,Night (The Night Trilogy #1),Nonfiction,1,Night is a work by Elie Wiesel about his exper...,£13.51,3,http://books.toscrape.com/catalogue/unicorn-tr...,http://books.toscrape.com/media/cache/b2/45/b2...


In [19]:
# Save df to csv
df.to_csv('books.csv', sep=',', encoding='utf-8', index=False)